In [1]:
import glob

import numpy as np
import pandas as pd

import xarray as xr

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [6]:
df = pd.read_parquet('../data/projections_gares/')
df['tasmaxAdjust'] = df['tasmaxAdjust'] - 273.15
df['year'] = df['time'].dt.year
df = df.sort_values(by=['time', 'lat', 'lon'])
df

,time,tasmaxAdjust,lon,lat,gare,year
994,2015-06-01 12:00:00,29.417877,1.9421,42.433,Bourg-Madame,2015
963,2015-06-01 12:00:00,24.430481,3.1584,42.436,Cerbère,2015
1057,2015-06-01 12:00:00,27.631195,1.9106,42.454,Latour-de-Carol - Enveitg,2015
1059,2015-06-01 12:00:00,25.184479,2.0322,42.457,Saillagouse,2015
1128,2015-06-01 12:00:00,24.553925,2.0311,42.480,Font-Romeu-Odeillo-Via,2015
...,...,...,...,...,...,...
8027214,2099-08-31 12:00:00,22.738495,2.4166,50.960,Bergues,2099
8027226,2099-08-31 12:00:00,22.511292,2.1311,50.977,Gravelines,2099
8027248,2099-08-31 12:00:00,22.586365,2.3078,51.003,Grande-Synthe,2099
8027261,2099-08-31 12:00:00,22.658966,2.3778,51.027,Coudekerque-Branche,2099


In [8]:
df['lag_1'] = df['tasmaxAdjust'].shift(1)
df['lag_2'] = df['tasmaxAdjust'].shift(2)

mask_cond = (
    (df['tasmaxAdjust'] > 40) &
    (df['lag_1'] > 35) &
    (df['lag_2'] > 35)
)
df_cond_day = df[mask_cond]
df_cond_day.to_csv("../data/projection_tasmax_cond.csv", index=False)

df_cond_day = df_cond_day.drop(['lag_1', 'lag_2', 'year'], axis=1)

df_cond_day

,time,tasmaxAdjust,lon,lat,gare
2543620,2017-06-30 12:00:00,40.267120,4.8625,43.997,Sorgues - Châteauneuf-du-Pape
2543622,2017-06-30 12:00:00,40.403168,4.9249,43.997,Entraigues-sur-la-Sorgue
2544199,2017-06-30 12:00:00,40.510590,4.8938,44.042,Bédarrides
2544202,2017-06-30 12:00:00,40.940979,4.9875,44.042,Monteux
2544204,2017-06-30 12:00:00,40.945343,5.0500,44.042,Carpentras
...,...,...,...,...,...
6663017,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors Canal
6663018,2099-08-16 12:00:00,40.128906,4.8000,45.593,Chasse-sur-Rhône
6664596,2099-08-16 12:00:00,40.311615,0.1658,45.656,Angoulême
7423190,2099-08-25 12:00:00,40.520844,6.4704,43.445,Les Arcs - Draguignan


In [9]:
path_cond_min = "../data/projection_tasmin_cond.csv"
df_cond_night = pd.read_csv(path_cond_min)
df_cond_night['time'] = pd.to_datetime(df_cond_night['time'], yearfirst=True)
df_cond_night.drop(['Unnamed: 0', 'index', 'two_consecutive_nights', 'lat_y', 'lon_y'], axis=1, inplace=True)
df_cond_night.rename(columns={'lon_x': 'lon', 'lat_x': 'lat'}, inplace=True)
df_cond_night

,time,tasminAdjust,lon,lat,gare
0,2015-06-25 12:00:00,20.525177,3.1584,42.436,Cerbère
1,2015-07-09 12:00:00,22.949799,3.1584,42.436,Cerbère
2,2015-07-10 12:00:00,21.773010,3.1584,42.436,Cerbère
3,2015-07-11 12:00:00,20.451600,3.1584,42.436,Cerbère
4,2015-07-12 12:00:00,20.744904,3.1584,42.436,Cerbère
...,...,...,...,...,...
1292508,2099-08-14 12:00:00,21.571198,2.3778,51.027,Dunkerque
1292509,2099-08-15 12:00:00,20.573029,2.3778,51.027,Coudekerque-Branche
1292510,2099-08-15 12:00:00,20.573029,2.3778,51.027,Dunkerque
1292511,2099-08-16 12:00:00,20.912811,2.3778,51.027,Coudekerque-Branche


In [12]:
df_cond = pd.merge(
    left=df_cond_day, right=df_cond_night, how='left', on=['time', 'lon', 'lat', 'gare']
)
df_cond = df_cond.dropna()
df_cond.to_csv("../data/projection_cond_globale.csv")

df_cond

,time,tasmaxAdjust,lon,lat,gare,tasminAdjust
0,2017-06-30 12:00:00,40.267120,4.8625,43.997,Sorgues - Châteauneuf-du-Pape,22.751709
1,2017-06-30 12:00:00,40.403168,4.9249,43.997,Entraigues-sur-la-Sorgue,21.857330
2,2017-06-30 12:00:00,40.510590,4.8938,44.042,Bédarrides,22.799774
3,2017-06-30 12:00:00,40.940979,4.9875,44.042,Monteux,22.024414
4,2017-06-30 12:00:00,40.945343,5.0500,44.042,Carpentras,20.589417
...,...,...,...,...,...,...
55000,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors,21.810760
55001,2099-08-16 12:00:00,40.459778,4.7679,45.593,Givors Canal,21.810760
55002,2099-08-16 12:00:00,40.128906,4.8000,45.593,Chasse-sur-Rhône,22.124450
55003,2099-08-16 12:00:00,40.311615,0.1658,45.656,Angoulême,21.228088


In [ ]:
trac_dates_1 = range(2042, 2062)
trac_dates_2 = range(2068, 2088)

df_trac_1 = df_cond[df_cond['year'].isin(trac_dates_1)]
df_trac_2 = df_cond[df_cond['year'].isin(trac_dates_2)]

df_trac_1 = df_trac_1.groupby(['year', 'gare', 'lat', 'lon'])['tasmaxAdjust'].count().reset_index()
df_trac_2 = df_trac_2.groupby(['year', 'gare', 'lat', 'lon'])['tasmaxAdjust'].count().reset_index()

df_trac_1